In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import sys
import mnist
import matplotlib.pyplot as plt
import numpy as np
import datetime
from scipy.ndimage.interpolation import shift

sys.path.append('mnist')

In [17]:
def load_data():
    train_images = mnist.train_images()
    train_labels = mnist.train_labels()

    test_images = mnist.test_images()
    test_labels = mnist.test_labels()

    train_images = train_images / 255.0
    test_images = test_images / 255.0

    x_train_shift = []
    y_train_augmented = []
    for dx, dy in ((-1, -1), (0, -1), (1, -1), (-1, 0), (0, 0), (1, 0), (-1, 1), (0, 1), (1, 1)):
        for image, label in zip(train_images, train_labels):
            x_train_shift.append(shift_image(image, dx, dy))
            y_train_augmented.append(label)
    x_train_shift = np.array(x_train_shift)
    y_train_augmented = np.array(y_train_augmented)

    print(x_train_shift.shape, y_train_augmented.shape, test_images.shape, test_labels.shape)

    return x_train_shift, y_train_augmented, test_images, test_labels

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((28, 28, 1), name='reshape1'),
        tf.keras.layers.Conv2D(8, [3, 3], strides=1, padding='SAME', name='conv1',
                               input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=[3, 3], strides=2, name='pool1'),
        tf.keras.layers.Conv2D(12, [3, 3], strides=1, padding='SAME', name='conv2'),
        tf.keras.layers.MaxPooling2D(pool_size=[3, 3], strides=2, name='pool2'),
        tf.keras.layers.Conv2D(16, [3, 3], strides=1, padding='SAME', name='conv3'),
        tf.keras.layers.MaxPooling2D(pool_size=[3, 3], strides=2, name='pool3'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu, name='fc1'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(100, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dx, dy], cval=0, mode='constant')
    return shifted_image

In [18]:
X_train, Y_train, X_test, Y_test = load_data()
# X_train = mnist.train_images() / 255.0
# Y_train = mnist.train_labels()
# X_test = mnist.test_images() / 255.0
# Y_test = mnist.test_labels()

model = create_model()


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


log_dir=".\\logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(X_train, Y_train, epochs=5, validation_data=(X_test, Y_test), callbacks=[tensorboard_callback])
test_loss, test_acc = model.evaluate(X_test, Y_test)
print("Test Acc = ", test_acc)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 36s 599us/sample - loss: 0.2766 - acc: 0.9132 - val_loss: 0.0854 - val_acc: 0.9724
Epoch 2/5
60000/60000 [==============================] - 36s 608us/sample - loss: 0.0976 - acc: 0.9704 - val_loss: 0.0530 - val_acc: 0.9835
Epoch 3/5
60000/60000 [==============================] - 37s 609us/sample - loss: 0.0751 - acc: 0.9771 - val_loss: 0.0449 - val_acc: 0.9860
Epoch 4/5
60000/60000 [==============================] - 37s 611us/sample - loss: 0.0612 - acc: 0.9813 - val_loss: 0.0458 - val_acc: 0.9849
Epoch 5/5
10000/10000 [==============================] - 1s 103us/sample - loss: 0.0497 - acc: 0.9845
Test Acc =  0.9845


In [ ]:
%tensorboard --logdir ./logs/fit

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis = 1)

In [ ]:
y_error = {}
n_error = []
for i in range(len(y_pred)):
    if y_pred[i] != Y_test[i]:
        n_error.append(i)
        if Y_test[i] in y_error.keys():
            y_error[Y_test[i]] += 1
        else:
            y_error[Y_test[i]] = 1

In [ ]:
for i in range(10):
    print(i, ":", y_error[i])

In [ ]:
fig, ax = plt.subplots()
ax.imshow(X_test[100])

In [ ]:
plt.figure(1, (10,10))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
for i in range(4):
    for j in range(4):
        n = n_error[i*4+j]
        ax = plt.subplot(4, 4, i*4+j+1)
        ax.set_title("pred:" + str(y_pred[n]) +" acc:"+ str(Y_test[n]))
        ax.imshow(X_test[n])

In [ ]:
x_shift_test = []
for dx, dy in ((-2, -2), (0, -2), (2, -2), (-2, 0), (0, 0), (2, 0), (-2, 2), (0, 2), (2, 2)):
    x_shift_test.append(shift_image(X_train[100], dx, dy))

plt.figure(1, (10,10))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
for i in range(3):
    for j in range(3):
        n = i*3+j
        ax = plt.subplot(3, 3, i*3+j+1)
        ax.imshow(x_shift_test[n])